In [271]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import datetime


def duplicates_by(df, columns):
    return df[df.duplicated(subset=columns,keep=False)].sort_values(columns)


%matplotlib inline
#os.chdir('/Users/flatironschooldc3/FlatironSchoolRepo/dc-ds-111819/module-1/project/unzipped_data')
os.chdir('/Users/flatironschooldc/Code/LearnCo/dc-ds-111819/module-1/project/data/')

In [341]:
#Rotten Tomatoes
rtm = pd.read_csv("rt.movie_info.tsv", sep='\t') 

# TheMovieDB (TMDB)
tmdbm = pd.read_csv("tmdb.movies.csv")
tmdbb = pd.read_csv("tn.movie_budgets.csv")


In [342]:
#-investigation commands-
#tmdbm.isna().sum()
#tmdbb.isna().sum()

#tmdbm.original_language.value_counts()


#-flag outliers-

# z = np.abs(stats.zscore(tmdbb[col]))
#     threshold = 4.26 #this corresponds to a probability below 1:10,000
#     outliers = np.where(z>threshold)

#-look at duplicates-
#tmdbb.loc[tmdbb['id']==100]
#duplicates_by(tmdbm,['title'])
#duplicates_by(tmdbb,['title'])
#duplicates_by(tmdbm,['title','release_date'])
#duplicates_by(tmdbb,['title','release_date'])

#-format of merge column-
#tmdbb.release_date.head()
#tmdbm.release_date.head()

#print(tmdbb.sort_values('title').title.head(25))
#print(tmdbm.sort_values('title').title.head(25))

#tmdbb[tmdbb['title'].str.contains("Potter")]
#tmdbm[tmdbm['title'].str.contains("Potter")]

#-confirm merge-
#tmd['_merge'].value_counts()

#Cleaning commands

#remove duplicates
tmdbm.drop_duplicates(subset=['title','release_date'], inplace=True)
tmdbb.drop_duplicates(inplace=True)

#drop pointless cols
del_cols = ['Unnamed: 0','genre_ids'] 
for col in del_cols:
    del tmdbm[col]

#flag title changes
tmdbm['title_change'] = tmdbm['original_title'] != tmdbm['title']

#rename columns
tmdbb.rename(columns={'movie':'title'},inplace=True)

#convert dollars to numeric and flag outliers
for col in ['production_budget','domestic_gross','worldwide_gross']:
    tmdbb[col] = tmdbb[col].apply(lambda x:float(x[1:].replace(',','')))
    
#standardize dates
tmdbb['release_date'] = pd.to_datetime(tmdbb['release_date'],infer_datetime_format=True)
tmdbm['release_date'] = pd.to_datetime(tmdbm['release_date'],infer_datetime_format=True)
    
#calculate ROI variables
tmdbb['domestic_roi'] = (tmdbb['domestic_gross']/tmdbb['production_budget'])
tmdbb['worldwide_roi'] = (tmdbb['worldwide_gross']/tmdbb['production_budget'])


tmd = pd.merge(tmdbb,tmdbm,on=['title','release_date'],indicator=False,validate="1:1")


In [344]:
#do some sanity checks
tmd['worldwide_roi'].describe()

count    1270.000000
mean        2.084077
std         7.307139
min         0.000000
25%         0.385403
50%         0.967613
75%         2.016782
max       227.644100
Name: domestic_roi, dtype: float64

In [345]:
tmd['domestic_roi'].describe()

count    1270.000000
mean        2.084077
std         7.307139
min         0.000000
25%         0.385403
50%         0.967613
75%         2.016782
max       227.644100
Name: domestic_roi, dtype: float64

In [346]:
tmd.head()

,id_x,release_date,title,production_budget,domestic_gross,worldwide_gross,domestic_roi,worldwide_roi,id_y,original_language,original_title,popularity,vote_average,vote_count,title_change
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,1.789430,6.532577,19995,en,Avatar,26.526,7.4,18676,False
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,0.587101,2.546673,1865,en,Pirates of the Caribbean: On Stranger Tides,30.579,6.4,8571,False
2,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.388403,4.243841,99861,en,Avengers: Age of Ultron,44.383,7.3,13457,False
3,7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,2.262718,6.827114,299536,en,Avengers: Infinity War,80.773,8.3,13948,False
4,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,0.763414,2.186484,141052,en,Justice League,34.953,6.2,7510,False


In [349]:
tmd_test = tmd[['production_budget',
                'domestic_gross',
                'worldwide_gross',
                'domestic_roi',
                'worldwide_roi',
                'popularity',
                'vote_average',
                'vote_count']]


In [350]:
tmd_test.corr()

,production_budget,domestic_gross,worldwide_gross,domestic_roi,worldwide_roi,popularity,vote_average,vote_count
production_budget,1.000000,0.723722,0.796485,-0.088761,-0.052125,0.618293,0.170412,0.657029
domestic_gross,0.723722,1.000000,0.942370,0.059798,0.089948,0.616461,0.315010,0.790509
worldwide_gross,0.796485,0.942370,1.000000,0.021066,0.073495,0.638779,0.287215,0.790356
domestic_roi,-0.088761,0.059798,0.021066,1.000000,0.986319,0.018369,0.015318,0.037421
worldwide_roi,-0.052125,0.089948,0.073495,0.986319,1.000000,0.056300,0.028488,0.082226
popularity,0.618293,0.616461,0.638779,0.018369,0.056300,1.000000,0.352290,0.690631
vote_average,0.170412,0.315010,0.287215,0.015318,0.028488,0.352290,1.000000,0.429410
vote_count,0.657029,0.790509,0.790356,0.037421,0.082226,0.690631,0.429410,1.000000
